In [1]:
#-*- coding:utf-8 -*-
#'''
# Created on 18-10-15
#
# @Author: Greg Gao(laygin)
#'''
import os
import numpy as np
import cv2
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import imutils
from pixellink_model import create_pixellink_model
from pixellink_utils import *

Using TensorFlow backend.


In [2]:
r_mean = 123.
g_mean = 117.
b_mean = 104.
rgb_mean = [r_mean, g_mean, b_mean]


#img_path = './samples/TOP_29_121222.jpg'
save_weights = './weights/pixellink.h5'
model = create_pixellink_model(acf='relu')
model.load_weights(save_weights)

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
# 設定 影片 解析度
video_width = 640
video_height = 480

#cap = cv2.VideoCapture(0)   # device number 0
cap = cv2.VideoCapture(0)   # device number 1
cap.set(cv2.CAP_PROP_FRAME_WIDTH, video_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, video_height)
print("capture device is open: " + str(cap.isOpened()))

capture device is open: True


# While-loop 主程式

In [4]:
# loop to read image  
while True:

    # keyboard input value
    key = cv2.waitKey(1) & 0xFF

    # read from camera
    ret, frame = cap.read()

    if ret:  # success read
        image, *r = resize_image(frame)
        image_ori = image.copy()
        image = image[...,::-1] - rgb_mean
        image = np.expand_dims(image, axis=0)

        pixel_pos_scores, link_pos_scores = model.predict(image)
        pixel_pos_scores = softmax(pixel_pos_scores, axis=-1)
        link_pos_scores_reshaped = link_pos_scores.reshape(link_pos_scores.shape[:-1]+(8, 2))
        link_pos_scores = softmax(link_pos_scores_reshaped, axis=-1)

        #masks = decode_batch(pixel_pos_scores, link_pos_scores, pixel_conf_threshold=0.75, link_conf_threshold=0.9)
        masks = decode_batch(pixel_pos_scores, link_pos_scores, pixel_conf_threshold=0.75, link_conf_threshold=0.98)

        bboxes = mask_to_bboxes(masks[0], image_ori.shape, min_area=300, min_height=30)
        
        
        image_c = image_ori.copy()
        for box in bboxes:
            points = np.reshape(box, [4, 2])
            cv2.line(image_c,tuple(points[0]),tuple(points[1]),(0,0,255),2)
            cv2.line(image_c,tuple(points[0]),tuple(points[3]),(0,0,255),2)
            cv2.line(image_c,tuple(points[1]),tuple(points[2]),(0,0,255),2)
            cv2.line(image_c,tuple(points[2]),tuple(points[3]),(0,0,255),2)
        
        #if start_video:     # start
        #    out.write(frame)
        #    frame_count += 1

    
        #cv2.putText(frame, str(frame_count) , (40, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL,1, (255, 0, 255), 1, cv2.LINE_AA)
        cv2.imshow('frame', frame)
        cv2.imshow('detect result', image_c)
        
    else:
        print('camera read fail.')
        break

    #if frame_count > FRAME_NUM:   # FPS: 30, so 30秒產生 900張
    #    break
    if key == ord('q'):     # press 'q' to leave while
        break
    #if key == ord(' '):     # press 's' to start
    #    print('Starting capture video...')
    #    start_video = True

# release VideoWriter object clsoe all windows
#out.release()
cap.release()
cv2.destroyAllWindows()